In [23]:
import os, glob
import csv
import pydicom
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from tqdm import tqdm
import shutil
from distutils.dir_util import copy_tree

In [11]:
text_file = 'same_slice_size_cases_path.txt'
file1 = open(text_file, 'r')
Lines = file1.readlines()
same_patient_ids = []
for line in Lines:
    same_patient_ids.append(line.split(',')[1].split('/')[1])
    # print(line.split(',')[1].split('/')[1])
print(same_patient_ids)

['HCC_1104', 'HCC_1105', 'HCC_1106', 'HCC_1107', 'HCC_1109', 'HCC_1110', 'HCC_1111', 'HCC_1114', 'HCC_1115', 'HCC_1116', 'HCC_1119', 'HCC_1120', 'HCC_1121', 'HCC_1122', 'HCC_1126', 'HCC_1128', 'HCC_1130', 'HCC_1131', 'HCC_1469', 'HCC_1272', 'HCC_1274', 'HCC_1275', 'HCC_1276', 'HCC_1277', 'HCC_1282', 'HCC_1284', 'HCC_1285', 'HCC_1289', 'HCC_1297', 'HCC_1298', 'HCC_1302', 'HCC_1304', 'HCC_1305', 'HCC_1307', 'HCC_1360', 'HCC_1132', 'HCC_1160', 'HCC_1199', 'HCC_1232', 'HCC_1270', 'HCC_1577', 'HCC_1589', 'HCC_1208', 'HCC_1209', 'HCC_1228', 'HCC_1229', 'HCC_1230', 'HCC_1765', 'HCC_1408', 'HCC_1161', 'HCC_1162', 'HCC_1163', 'HCC_1165', 'HCC_1170', 'HCC_1171', 'HCC_1172', 'HCC_1173', 'HCC_1174', 'HCC_1175', 'HCC_1176', 'HCC_1177', 'HCC_1178', 'HCC_1179', 'HCC_1181', 'HCC_1182', 'HCC_1269', 'HCC_1444', 'HCC_1133', 'HCC_1134', 'HCC_1136', 'HCC_1139', 'HCC_1141', 'HCC_1143', 'HCC_1146', 'HCC_1148', 'HCC_1149', 'HCC_1150', 'HCC_1151', 'HCC_1152', 'HCC_1154']


In [14]:
data_root_pth = "/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom"

same_slice_size_paths = []
for patient_id in same_patient_ids:
    same_slice_size_paths.append(os.path.join(data_root_pth, patient_id))
print(same_slice_size_paths)

['/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1104', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1105', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1106', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1107', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1109', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1110', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1111', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1114', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1115', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1116', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1119', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1120', '/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1121', '/media/monib/External D

In [18]:
all_d_paths = []
all_p_paths = []
for same_slice_size_path in same_slice_size_paths:
    phases_dirs = os.listdir(same_slice_size_path)
    # print(phases_dirs)
    for p in phases_dirs:
        if p == "D":
            all_d_paths.append(os.path.join(same_slice_size_path, p))
        if p == "P" or p=="V":
            all_p_paths.append(os.path.join(same_slice_size_path, p))
for i in range(len(all_p_paths)):
    print(all_d_paths[i], all_p_paths[i])

/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1104/D /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1104/V
/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1105/D /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1105/P
/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1106/D /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1106/V
/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1107/D /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1107/V
/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1109/D /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1109/V
/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1110/D /media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1110/V
/media/monib/External Disk/work2022/Base_Dataset/KTL_HCC_Dicom/HCC_1111/D /media/monib/External Disk/work2022/Ba

In [27]:
dest_root_path ="/media/monib/External Disk/work2022/Base_Dataset/KTL_Same_Slice_D_P"


for d_path, p_path in zip(all_d_paths, all_p_paths):
    print(d_path.split("/")[-2])
    save_path_d = os.path.join(dest_root_path, f"delayed/{d_path.split('/')[-2]}" +"_d")
    save_path_p = os.path.join(dest_root_path, f"portal/{p_path.split('/')[-2]}" +"_p")
    os.makedirs(save_path_d, exist_ok=True)
    os.makedirs(save_path_p, exist_ok=True)
    copy_tree(d_path, save_path_d)
    copy_tree(p_path, save_path_p)
print(f"Copied {len(all_d_paths)} data")


HCC_1104
HCC_1105
HCC_1106
HCC_1107
HCC_1109
HCC_1110
HCC_1111
HCC_1114
HCC_1115
HCC_1116
HCC_1119
HCC_1120
HCC_1121
HCC_1122
HCC_1126
HCC_1128
HCC_1130
HCC_1131
HCC_1469
HCC_1272
HCC_1274
HCC_1275
HCC_1276
HCC_1277
HCC_1282
HCC_1284
HCC_1285
HCC_1289
HCC_1297
HCC_1298
HCC_1302
HCC_1304
HCC_1305
HCC_1307
HCC_1360
HCC_1132
HCC_1160
HCC_1199
HCC_1232
HCC_1270
HCC_1577
HCC_1589
HCC_1208
HCC_1209
HCC_1228
HCC_1229
HCC_1230
HCC_1765
HCC_1408
HCC_1161
HCC_1162
HCC_1163
HCC_1165
HCC_1170
HCC_1171
HCC_1172
HCC_1173
HCC_1174
HCC_1175
HCC_1176
HCC_1177
HCC_1178
HCC_1179
HCC_1181
HCC_1182
HCC_1269
HCC_1444
HCC_1133
HCC_1134
HCC_1136
HCC_1139
HCC_1141
HCC_1143
HCC_1146
HCC_1148
HCC_1149
HCC_1150
HCC_1151
HCC_1152
HCC_1154
Copied 80 data
